<a href="https://colab.research.google.com/github/ThisIsFarhan/Language_Models/blob/main/4_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain>=0.1.17 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 langchain_community
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69

ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 14.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [2]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-02-06 12:57:28--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 3.165.160.61, 3.165.160.11, 3.165.160.12, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.61|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1738850248&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczODg1MDI0OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg4Z

In [3]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

ModuleNotFoundError: No module named 'langchain_community'

In [ ]:
from langchain import PromptTemplate

#Chains

In [ ]:
#Single chain


template = """<s><|User|>
{input_prompt}<|END|>
<|Assistant|>
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

basic_chain = prompt | llm
basic_chain.invoke({
    "input_prompt":"Tell me about computer engineering"
})

In [ ]:
#Multiple chain
from langchain import LLMChain

#--------------------Templates------------------------------
summary_template = """<s><|user|>
Create a title for a story about {summary}. Only return the one-liner title.<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(template=summary_template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=summary_prompt, output_key="title")

title_template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(
    template=title_template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=title_prompt, output_key="character")

story_template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=story_template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")


#Defining the chain
llm_chain = title | character | story


#invokation
llm_chain.invoke("A person developing an efficient AI agent")

#Memory

In [ ]:
#Conversation Buffer

from langchain import LLMChain
from langchain.memory import ConversationBufferMemory

template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)


In [ ]:
llm_chain.invoke({"input_prompt":"Hi I am Farhan"})

In [ ]:
llm_chain.invoke({"input_prompt":"Whats my name?"})

In [ ]:
#Conversation Buffer Window
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [ ]:
llm_chain.invoke({"input_prompt":"Hi My name is Farhan"})

In [ ]:
llm_chain.invoke({"input_prompt":"Tell me a joke"})

In [ ]:
llm_chain.invoke({"input_prompt":"What is 2+2?"})

In [ ]:
llm_chain.invoke({"input_prompt":"What is the mean of AGI?"})

In [ ]:
llm_chain.invoke({"input_prompt":"what is my name?"})

In [ ]:
#Summary Buffer
from langchain.memory import ConversationSummaryMemory
summary_prompt_template = """<s><|user|>Summarize the conversation and update it with new lines

Currenty Summary:
{summary}

New Lines of conversation:
{new_lines}

New Summary:<|end|>
<|assistant|>"""

summary_prompt = PromptTemplate(
    llm = llm,
    template=summary_prompt_template,
    input_variables=["new_lines","summary"]
)

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",prompt=summary_prompt)

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [ ]:
llm_chain.invoke({"input_prompt":"Hi My name is Farhan"})

In [ ]:
llm_chain.invoke({"input_prompt":"What is MQTT?"})

In [ ]:
llm_chain.invoke({"input_prompt":"What is AGI?"})

In [ ]:
llm_chain.invoke({"input_prompt":"What is my name?"})

#ReAct Agent

In [4]:
!pip install langchain_groq duckduckgo-search>=5.2.2 langchain>=0.1.17 langchain_community

In [5]:
from google.colab import userdata
from langchain_groq import ChatGroq

groq_api_key=userdata.get('groq_api_key')
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")

In [6]:
from langchain import PromptTemplate

react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(template=react_template, input_variables=["tools", "tool_names","input","agent_scratchpad"])

In [8]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
search_tool = Tool(name="ducksearch",description="A web search engine. Use this to as a search engine for general queries.",func=search.run)
tools = load_tools(["llm-math"],llm=llm)
tools.append(search_tool)

In [9]:
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question

In [10]:
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handling_parsing_errors=True)


In [12]:
agent_executor.invoke(
    {
        "input":"Tell me what KiloGrams actually means and convert 2.45kg to grams"
    }
)



> Entering new AgentExecutor chain...
Thought: I need to look up the definition of Kilograms and how it relates to grams. Then I can use a calculator to perform the conversion. 
Action: ducksearch
Action Input: What is a Kilogram? How many grams are in a kilogram?snippet: One kilogram is equal to 1,000 grams. In the metric system, the unit of mass is the gram (g), and the prefix "kilo-" denotes a factor of 1000. Therefore, when converting from kilograms to grams, 1 kilogram is equivalent to 1000 grams. The conversion can be expressed mathematically as follows: 1 kilo, title: 1 kilogram is equal to grams. - GeeksforGeeks, link: https://www.geeksforgeeks.org/1-kilogram-is-equal-to-grams/, snippet: One kilogram is equal to 1000 grams, creating a ratio of 1:1000 between the two units. This means that one kilogram is 1000 times larger than one gram. For example, if you have 500 grams, it would be equivalent to 0.5 kilograms since you divide the grams by 1000. Conversely, if you have 2 kil

{'input': 'Tell me what KiloGrams actually means and convert 2.45kg to grams',
 'output': '2.45 kilograms is equal to 2450 grams.'}